# MNIST Image Classification with TensorFlow

This notebook demonstrates how to implement different image models on MNIST using the [tf.keras API](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras).

## Learning Objectives
1. Understand how to build a Dense Neural Network (DNN) for image classification
2. Understand how to use dropout (DNN) for image classification
3. Understand how to use Convolutional Neural Networks (CNN)


In [ ]:
!sudo chown -R jupyter:jupyter /home/jupyter/training-data-analyst

In [ ]:
from datetime import datetime
import os
import shutil

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.layers import (
    Conv2D, Dense, Dropout, Flatten, MaxPooling2D, Softmax)

## Write Input Functions

As usual, we need to specify input functions for training and evaluating. We'll scale each pixel value so it's a decimal value between 0 and 1 as a way of normalizing the data.



In [ ]:
import tensorflow as tf

def scale(image, label):
    """Scales images from a 0-255 int range to a 0-1 float range"""
    image = tf.cast(image, tf.float32)
    image /= 255
    image = tf.expand_dims(image, -1)
    return image, label


def load_dataset(
        data, training=True, buffer_size=5000, batch_size=100, nclasses=10):
    """Loads MNIST dataset into a tf.data.Dataset"""
    (x_train, y_train), (x_test, y_test) = data
    x = x_train if training else x_test
    y = y_train if training else y_test
    # One-hot encode the classes
    y = tf.keras.utils.to_categorical(y, nclasses)
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.map(scale).batch(batch_size)
    if training:
        dataset = dataset.shuffle(buffer_size).repeat()
    return dataset


Next, let's code the models! The [tf.keras API](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras) accepts an array of [layers](https://www.tensorflow.org/api_docs/python/tf/keras/layers) into a [model object](https://www.tensorflow.org/api_docs/python/tf/keras/Model), so we can create a dictionary of layers based on the different model types we want to use. The below file has two functions: `get_layers` and `build_model`. We will build the structure of our model in `get_layers`. 

**TODO 1**: Define the Keras layers for a linear model   
**TODO 2**: Define the Keras layers for a DNN model with dropouts  
**TODO 3**: Define the Keras layers for a CNN model  

Hint: These models progressively build on each other. Look at the imported `tensorflow.keras.layers` modules and the default values for the variables defined in `get_layers` for guidance.

In [ ]:
# Image Variables
WIDTH = 28
HEIGHT = 28

def get_layers(
        model_type,
        nclasses=10,
        hidden_layer_1_neurons=400,
        hidden_layer_2_neurons=100,
        dropout_rate=0.25,
        num_filters_1=64,
        kernel_size_1=3,
        pooling_size_1=2,
        num_filters_2=32,
        kernel_size_2=3,
        pooling_size_2=2):
    """Constructs layers for a keras model based on a dict of model types."""
    model_layers = {
        'linear': [
        # TODO 1
        ],
        'dnn': [
        # TODO 2
        ],
        'cnn': [
        # TODO 3
        ]
    }
    return model_layers[model_type]


def build_model(layers):
    """Compiles keras model for image classification."""
    model = Sequential(layers)
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model


## Training

With everything set up, let's run train code. 

In [ ]:
MODEL_TYPE = ["linear", "dnn", "cnn"]
model_results = {}
EPOCHS = 10
STEPS_PER_EPOCH = 100

mnist = tf.keras.datasets.mnist.load_data()
train_data = load_dataset(mnist)
validation_data = load_dataset(mnist, training=False)

for model in MODEL_TYPE:
    print('Start {} Training'.format(model))
    model_layers = get_layers(model)
    image_model = build_model(model_layers)
    history = image_model.fit(
        train_data,
        validation_data=validation_data,
        epochs=EPOCHS,
        steps_per_epoch=STEPS_PER_EPOCH)
    
    model_results.update({model: history})

In [ ]:
for model in MODEL_TYPE:
    print('accuracy of {}: {}'.format(model, model_results[model].history['accuracy'][-1]))

## predict and visualize results

In [ ]:
test_ds = load_dataset(
    mnist,
    training=False,
    batch_size=200,
)
image, label_ohe = next(iter(test_ds))

In [ ]:
label = np.argmax(label_ohe, axis=1)
linear_res = np.argmax(model_results['linear'].model.predict(image), axis=1)
dnn_res = np.argmax(model_results['dnn'].model.predict(image), axis=1)
cnn_res = np.argmax(model_results['cnn'].model.predict(image), axis=1)

for i, im in enumerate(image):
    # visualize images which were mispredicted.
    if label[i] != linear_res[i] or label[i] != dnn_res[i] or label[i] != cnn_res[i]:
        MSG = 'label:{}, linear:{}, dnn:{}, cnn:{}'
        print(MSG.format(label[i], linear_res[i], dnn_res[i], cnn_res[i]))
        plt.imshow(im)
        plt.show()

Copyright 2020 Google Inc.
Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at
http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.